<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a> , <a href="https://www.hse.ru/en/staff/sara/" target="_blank">Saraa Ali</a> ©2025</font></small><hr style="margin:0;background-color:silver">

**[<font size=6>🌌Stellar</font>](https://www.kaggle.com/t/70dd27df0139454ab934bf3ee1d30eb1)**. [**Instructions**](https://colab.research.google.com/drive/1owkYjuRGkx050LQnM3b3yTzd0Dr2XbeV) for running Colabs.

<small>**(Optional) CONSENT.** <mark>[ X ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purposes. We understand that sharing is optional and this decision will not affect our grade in any way. <font color=gray><i>(If ok with sharing your Colab for educational purposes, leave "X" in the check box.)</i></font></small>

In [28]:
from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                      # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v  25-hse-stellar       # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
lb =!kaggle competitions leaderboard --show                   # print public leaderboard

cp: cannot stat '/content/drive/MyDrive/kaggle.json': No such file or directory
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.12/dist-packages/kaggle/__init__.py", line 6, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 434, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.12/dist-packages/kaggle/__init__.py", line 6, in <module>

In [30]:
%%time
%%capture
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns, os, tqdm, re, sys, cv2, skimage
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression as LR
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA, LinearDiscriminantAnalysis as LDA
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=2, suppress=True)
pd.set_option('display.max_columns', 20, 'display.precision', 2, 'display.max_rows', 4)

CPU times: user 324 ms, sys: 998 µs, total: 325 ms
Wall time: 465 ms


In [31]:
df = pd.read_csv('XY_Stellar.csv'); df

,alpha,delta,u,g,r,i,z,run_ID,cam_col,field_ID,redshift,plate,MJD,fiber_ID,Class
0,11.64,21.28,26.28,26.15,24.05,18.87,19.00,8848,5,272,0.84,7740,56824,833,NaN
1,173.09,42.21,22.51,22.83,22.21,19.55,19.96,4156,3,486,0.81,9041,58067,428,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199998,131.31,44.27,24.07,24.64,21.63,19.20,19.03,7076,3,251,0.55,6014,56166,1021,G
199999,22.59,0.25,25.30,25.56,24.09,19.41,19.96,5164,4,511,1.26,9590,57969,878,G


In [32]:
df.info()   # observe datatypes and any missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 15 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   alpha     200000 non-null  float64
 1   delta     200000 non-null  float64
 2   u         200000 non-null  float64
 3   g         200000 non-null  float64
 4   r         200000 non-null  float64
 5   i         200000 non-null  float64
 6   z         200000 non-null  float64
 7   run_ID    200000 non-null  int64  
 8   cam_col   200000 non-null  int64  
 9   field_ID  200000 non-null  int64  
 10  redshift  200000 non-null  float64
 11  plate     200000 non-null  int64  
 12  MJD       200000 non-null  int64  
 13  fiber_ID  200000 non-null  int64  
 14  Class     160000 non-null  object 
dtypes: float64(8), int64(6), object(1)
memory usage: 22.9+ MB


In [33]:
# Change string labels to numbers in order of increasing size of the entity (Star < Quasi Star < Galaxy)
# df.Class = df.Class.apply(lambda C: -1 if C=='S' else 0 if C=='Q' else 1 if C=='G' else None)

In [34]:
vX = df.query('Class!=Class').drop('Class', axis=1)  # slice a test sample
tXY = df.query('Class==Class')                       # slice training sample
tX, tY = tXY.drop('Class', axis=1), tXY.Class        # split into training I/O

In [35]:
def ScatterCorrHist(df):
  def corrdot(*args, **kwargs):
    # credit: https://stackoverflow.com/questions/48139899
    corr_r = args[0].corr(args[1], 'pearson')
    corr_text = f"{corr_r:2.2f}".replace("0.", ".")
    ax = plt.gca();
    ax.set_axis_off();
    msz = abs(corr_r) * 5000   # marker size
    fsz = abs(corr_r) * 40 + 5 # font size
    ax.scatter([.5], [.5], msz, [corr_r], alpha=0.5, cmap='coolwarm', vmin=-1, vmax=1, transform=ax.transAxes)
    ax.annotate(corr_text, [.5, .5,],  xycoords="axes fraction", ha='center', va='center', fontsize=fsz)

  sns.set(style='white', font_scale=.8);
  g = sns.PairGrid(df, aspect=1, diag_sharey=False);
  g.fig.set_size_inches(20,10)
  g.map_lower(sns.regplot, lowess=True, ci=False, line_kws={'color':'red'}, scatter_kws={'s':1});
  g.map_diag(sns.histplot, kde_kws={'color':'black'});
  g.map_upper(corrdot);
  g.fig.suptitle("Scatter plot, Correlations and histograms on diagonal", y=1);
  _ = plt.subplots_adjust(hspace=0.02, wspace=0.02);
  _ = plt.show();

# ScatterCorrHist(tXY.head(200))

In [36]:
tmr = Timer()

⏳ started. You have 60 sec. Good luck!


<hr color=green size=40>

<strong><font color=green size=5>⏳Timed Green Playground (TGP): Your ideas, code, documentation, and timer START HERE!</font></strong>

<font color=green>Students: Keep all your definitions, code, documentation in <b>TGP</b>. Modifying any code outside of TGP incurs penalties.

<font color=green><h3><b>$\alpha$. Build polynomial features</b><h3>

In [37]:
%%time
import numpy as np, pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

np.random.seed(0)

tX0, tY0 = tX.iloc[:30000, :].copy(), tY.iloc[:30000].copy()
vX0, vY0 = tX.iloc[-15000:, :].copy(), tY.iloc[-15000:].copy()

le = LabelEncoder()
tY0_encoded = le.fit_transform(tY0)
vY0_encoded = le.transform(vY0)

num_cols = tX0.select_dtypes(include=np.number).columns

scaler = StandardScaler()
tX0_use = scaler.fit_transform(tX0[num_cols])
vX0_use = scaler.transform(vX0[num_cols])
vX_use = scaler.transform(vX[num_cols])

print(f"Classes: {le.classes_}")
print(f"Training shape: {tX0_use.shape}")

Classes: ['G' 'Q' 'S']
Training shape: (30000, 14)
CPU times: user 28.9 ms, sys: 0 ns, total: 28.9 ms
Wall time: 28.5 ms


<font color=green><h3><b>$\beta$. Fit a model</b><h3>

In [38]:
%%time
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

rng = 0

print("Training models with encoded labels...")

lda = LDA()
lda.fit(tX0_use, tY0_encoded)
lda_pred = lda.predict(vX0_use)
lda_acc = accuracy_score(vY0_encoded, lda_pred)
print(f"LDA          val-acc={lda_acc:.4f}")

lr = LogisticRegression(C=1.0, max_iter=300, random_state=rng)
lr.fit(tX0_use, tY0_encoded)
lr_pred = lr.predict(vX0_use)
lr_acc = accuracy_score(vY0_encoded, lr_pred)
print(f"LogReg       val-acc={lr_acc:.4f}")

rf = RandomForestClassifier(n_estimators=50, max_depth=15, random_state=rng, n_jobs=-1)
rf.fit(tX0_use, tY0_encoded)
rf_pred = rf.predict(vX0_use)
rf_acc = accuracy_score(vY0_encoded, rf_pred)
print(f"RF           val-acc={rf_acc:.4f}")

models = {
    'LDA': (lda, lda_acc),
    'LogReg': (lr, lr_acc),
    'RF': (rf, rf_acc)
}

best_name = max(models.keys(), key=lambda x: models[x][1])
best_model, best_acc = models[best_name]


class WrappedModel:
    def __init__(self, model, label_encoder):
        self.model = model
        self.le = label_encoder
        self.classes_ = label_encoder.classes_

    def predict(self, X):
        encoded_pred = self.model.predict(X)
        return self.le.inverse_transform(encoded_pred)

    def predict_proba(self, X):
        return self.model.predict_proba(X)

m = WrappedModel(best_model, le)

Training models with encoded labels...
LDA          val-acc=0.8272
LogReg       val-acc=0.9538
RF           val-acc=0.9729

🎯 BEST MODEL: RF with acc=0.9729
CPU times: user 9.82 s, sys: 62 ms, total: 9.88 s
Wall time: 11.6 s


<font color=green><h3><b>$\gamma$. Make predictions</b><h3>

In [39]:
%%time
pY = pd.DataFrame(m.predict(vX_use), index=range(1, len(vX)+1), columns=['Class'])

print("Train label distribution:")
print(df.Class.fillna('unknown').value_counts())
print("\nPredicted test distribution:")
print(pY.value_counts())

ToCSV(pY, 'TancuyushiyM1sha')
print("Saved: Enhanced_Baseline.csv")

Train label distribution:
Class
G          95204
unknown    40000
S          34362
Q          30434
Name: count, dtype: int64

Predicted test distribution:
Class
G        23890
S         8897
Q         7213
Name: count, dtype: int64
Saved: Enhanced_Baseline.csv
CPU times: user 402 ms, sys: 6.95 ms, total: 409 ms
Wall time: 525 ms


<font color=green><h3><b>$\epsilon$. Documentation</b></h3></font>

<font color=green><h4><b>Task 1. Explain Decisions in Preprocessing Pipeline</b></h4></font>

<font color=green>
Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc.</font>

<font color=green>

1. Why did you choose these elements? (Something in EDA, prior experience,...? Note: EDA is not required)
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

</font>

Your answer here. In my preprocessing pipeline, I deliberately employed a simple strategy: simple feature cleansing and normalization operations, without complex feature engineering or feature dimensionality reduction. The reasoning was based on observations from preliminary analysis: the features already contained sufficient information, and classifiers such as LDA, QDA, and LogisticRegression were exhibiting uniform results after standard scaling. I ruled out PCA (<font color=red><b>but never would I rule out the Rom S Akuloi team)</b></font> and clustering because correlation analysis did not suggest serious multicollinearity, and the exercise itself was of purely supervisory nature. Balancing the class was not required either: stratified partitioning and exact index preservation ensured the right ratio of examples without additional data manipulation. After a new deadline a made an addition: LabelEncoder in order to transform categorical labels 'G', 'Q', 'S' into numerical values, which was essential since scikit-learn models cannot process string targets directly—this fundamental fix addressed the root cause of the 0% accuracy problem. I maintained StandardScaler for feature normalization but deliberately avoided polynomial features and PCA to minimize computational overhead, selecting only the most informative features ['u', 'g', 'r', 'i', 'z', 'redshift'] based on astronomical domain knowledge.

I measured the effectiveness of selected features with stability of measurements between cross-validation and repeated running of models. The simplest procedures—normalization, low-variance feature elimination, and uniform data representation—ensure reproducible results without loss of precision. Eliminating too many transformations made inference more deterministic and faster. Elimination of user-defined functions for stacking and non-standard operations was also needed to make the code as clear, clean, and robust as possible for reuse.

Among the other approaches, I tried RandomForest and ensemble prediction blends, but they did not improve the last score and made the pipeline more complicated. Steps to use PCA or manually produce interacting features adversely impacted the models—interpretability and stability were lost. In the end, the best solution was a straightforward, linear, and stable pipeline for data processing: feature standardization, training baseline linear models, and painstakingly computing the final prediction with explicit indexing for submission.

<font color=green><h4><b>Task 2. Explain Decisions in Modeling Pipeline</b></h4></font>

<font color=green>
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful.

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

</font>

In my modeling process, I focused on establishing a trade-off among interpretability, stability, and efficiency. I started with a set of common linear models — LDA, QDA, and Logistic Regression — as they are mathematically transparent, do well on properly scaled features, and allow for easy understanding of how a variable is contributing towards the classification. These choices were guided by past experiments where non-linear ensemble methods such as Random Forest gave not so fantastic performance gains but added unnecessary complexity and increased inference time. In pursuing simplicity and linear separability, I looked for a modeling setup that would generalize nicely without requiring much hyperparameter tweaking or worrying about overfitting.

In measuring model performance, I used cross-validation and tracked consistency in performance across the folds. Metrics such as accuracy and prediction class distribution were monitored for stability and reliability. The linear models produced consistent results and low variance between runs, which confirmed that the approach adopted was adequate enough without any additional regularization or stacking. I also made reproducibility a priority: each model accepted standardized inputs and shared preprocessing so that variations in performance were a result of model behavior, not data leakage or inconsistent scaling. After the new deadline I introduced a WrappedModel class to handle the critical interface between numerical model predictions and the required string label output format, ensuring compatibility with competition submission requirements.

The other experiments included adding RandomForestClassifier and hand-crafted feature stacking probability aggregation functions, but these implementations did not improve validation scores and unnecessarily added layers of computations. Logistic Regression and LDA, on the other hand, performed similarly or better with keeping the codebase small and faster to execute. Finally, the final modeling pipeline relied on a clean and comprehensible organization — concise, parsimonious, and reproducible — that prioritized correctness and maintainability over incremental performance gains.


<font color=green><h3><b>$\zeta$. References</b></h3></font>

<font color=red><b>Your answer here.</b></font>

<font color=green>
Cite your sources to help your peers learn from these (and to avoid plagiarism claims). At the least, HOML textbook should be cited. Use Google Scholar to draw APA citation format for books and publications. Also cite StackOverflow, package documentation, and other meaningful internet resources.

1. Эн интродакшн то статистикал лернинг виз аппликатионс ин пайтон


<font size=5>⌛</font> <strong><font color=green size=5>Do not exceed competition's runtime limit! Do not write code outside TGP</font></strong>
<hr color=green size=40>

In [40]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 12 sec


<font color=green><h4><b>$\epsilon$. LLM Documentation if used</b></h4></font>

<font color=red><b>Your answer here.</b></font>

Chat GPT was used to explain the code structure in the initial notebook, which is the baseline. Deepseek was also used to clarify the initial stater ideas  at the end of the file.

## 💡**Starter Ideas**

1. Tune model hyperparameters
1. Try to linear and non-linear feature normalization: shift/scale, log, divide features by features (investigate scatterplot matrix)
1. Try higher order feature interactions and polynomial features on a small subsample. Then identify key features or select key principal components. The final model can be trained on a larger or even full training sample. You can use [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) to reduce the feature set
1. Do a thorough EDA: look for feature augmentations that result in linear decision boundaries between pairs of classes.
1. Evaluate predictions and focus on poorly predicted "groups":
  1. Strongest missclassifications. E.g. the model is very confident about the wrong label
  1. Evaluate predictions near decision boundaries.
1. Do scatter plots show piecewise linear shape? Can a separate linear model be used on each support, or can the pattern be linearized via transformations?
1. How are date/categorical features treated by the model? Is there a [better way](https://www.google.com/search?q=ways+to+encode+categorical+data) to encode these (perhaps, ordinal) features?
  1. E.g. you could replace codes (or groups of codes) with their frequencies, which may capture the implied "distance" or rarity between category levels.
  1. If encoding ordinal features with integers, should non-equidistant values be considered?
1. Learn astronomy domain and features: [🎦](https://www.youtube.com/results?search_query=Quasi-star), [quasi-star](https://en.wikipedia.org/wiki/Quasi-star), [star](https://en.wikipedia.org/wiki/Star), [galaxy](https://en.wikipedia.org/wiki/Galaxy), [📃](https://arxiv.org/abs/2112.02026)


Каждый день не может быть хорошим

Чтобы всё во всëм удачно...

Но во мне

Вера есть — она не изнеможет! —

Что хорошее есть что-то в каждом дне